In [1]:
# import necessary modules
import raadpy as rp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.colors import to_hex
from matplotlib.ticker import AutoMinorLocator
import numpy as np
import pandas as pd
import os
import csv

%matplotlib widget

In [2]:
# Make the cmnd list a dictionary
# Node Port Message : Description, # in graph, # in cmnd list

CMND_LIST = {
"4"  : {"14": { "-1" : "Emergency Power-off of PAYLOAD if OBC reboots,-1,33"}},

"12" : { 
    "4": {"8": "Suspend Payload as well PMT,12,36",
          "1": "Load Scenario Suspended, PMT PAYLOAD,12,25",
          "2": "Load Scenario Awaken, PMT PAYLOAD,11,27",
          "4": "Reboot PMT Payload,13,29",
          },

    "8": {"0F": "After an Emergency Shutdown happened to PMT,3,34",
          "00": "Load Scenario 0 (Default) PMT PAYLOAD,0,1",
          "01": "Load Scenario 1 PMT PAYLOAD,1,2",
          "02": "Load Scenario 2 PMT PAYLOAD,2,3",
          "03": "Load Scenario 3 PMT PAYLOAD,3,4",
          "04": "Load Scenario 4 PMT PAYLOAD,4,5",
          "05": "Load Scenario 5 PMT PAYLOAD,5,6",
          "06": "Load Scenario 6 PMT PAYLOAD,6,7",
          "07": "Load Scenario 7 PMT PAYLOAD,7,8",
          "08": "Load Scenario 8 PMT PAYLOAD,8,9",
          "09": "Load Scenario 9 PMT PAYLOAD,9,10",
          "10": "Load Scenario 10 PMT PAYLOAD,10,11"
          },

    "9": {"0F": "Load CUSTOM Scenario PMT PAYLOAD,14,23"},

    "1": {"00": "Ping PMT PAYLOAD,15,31"}
    },

"13" : { 
    "4": {"8": "Suspend Payload as well SiPM,12,37",
          "1": "Load Scenario Suspended, SiPM PAYLOAD,12,26",
          "2": "Load Scenario Awaken, SiPM PAYLOAD,11,28",
          "4": "Reboot SiPM Payload,13,30",
          },

    "8": {"0F": "After an Emergency Shutdown happened to SiPM,3,35",
          "00": "Load Scenario 0 (Default) SiPM PAYLOAD,0,12",
          "01": "Load Scenario 1 SiPM PAYLOAD,1,13",
          "02": "Load Scenario 2 SiPM PAYLOAD,2,14",
          "03": "Load Scenario 3 SiPM PAYLOAD,3,15",
          "04": "Load Scenario 4 SiPM PAYLOAD,4,16",
          "05": "Load Scenario 5 SiPM PAYLOAD,5,17",
          "06": "Load Scenario 6 SiPM PAYLOAD,6,18",
          "07": "Load Scenario 7 SiPM PAYLOAD,7,19",
          "08": "Load Scenario 8 SiPM PAYLOAD,8,20",
          "09": "Load Scenario 9 SiPM PAYLOAD,9,21",
          "10": "Load Scenario 10 SiPM PAYLOAD,10,22"
          },

    "9": {"0F": "Load CUSTOM Scenario SiPM PAYLOAD,14,24"},

    "1": {"00": "Ping SiPM PAYLOAD,15,32"}

}}

In [35]:
# call log file:
logfile = rp.log_expand("/home/raad/RAAD/Data/Light-1/light1-2022-09-15/light1-2022-09-15-se-log.txt")

In [32]:
def log_line_timestamp(logline:list,time:float=0):
    """Given a log file line as the lines decoded by log_expand, and a time
    return the time increment for this line

    Args:
        logline (list): The element of the decoded logfile corresponding to a line
        time (float): The time up to now

    Returns:
        time (float): Time increment after the execution of the command
    """

    splt = logline[0].split(' ')                        # Split the cmnd line (logline[0]) by the spaces 
    if "SE0>\n" not in splt[0]:                         # If the cmnd is not the end of the log file: 
        
        if "read" in splt[1]:                           # If cmnd is read time
            time = float (logline[1][0].split(" ")[3])  # Replace time with the time read

        if splt[1] == "delay":                          # If the cmnd is a delay
            time = time + (float(splt[2])/1000)         # Add time delay to previous time

        if splt[1] == "delayuntil":                     # If cmnd is delayuntil 
            time = float(splt[2])                       # Replace time by new time

        if "FAIL\n" in logline[1]:                      # if it was a payload cmnd and it failed (there will be delays)
            time = time + float(splt[4])                # take the time a payload cmnd takes to be excuted 

    return time

In [34]:
def log_with_timestamp(logfile:list):                        
    """Given a log file from the function log_expand
    return a list of log lines with the time increment of each line

    Args:
        logfile (list): a list of cubesat commands

    Returns:
        log_timestamp_list (list): a list of cubesat commands, each command with its timestamp
    """
    log_timestamp_list=[]                               # List to be returned later (cmnd, output, timestamp)
    time = 0                                            # Set initial time to zero, this step will not be needed
    for logline in logfile:
        time = log_line_timestamp(logline,time)
        log_timestamp_list.append ([logline,time])

    return log_timestamp_list

log_with_timestamp(logfile)


[[['SE0>script delayuntil 1663200000\n', ['OK\n']], 1663200000.0],
 [['SE0>csp txrx 4 14 600 000800201C\n',
   ['Answer: 0000\n',
    'Bytes: 2\n',
    'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n',
    'OK\n']],
  1663200000.0],
 [['SE0>csp txrx 4 14 600 000A00341C\n',
   ['Answer: 0000\n',
    'Bytes: 2\n',
    'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n',
    'OK\n']],
  1663200000.0],
 [['SE0>csp txrx 4 14 600 000700391C\n',
   ['Answer: 0000\n',
    'Bytes: 2\n',
    'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n',
    'OK\n']],
  1663200000.0],
 [['SE0>rtc read\n',
   ['curr rtc time 1663200000.012726 s, 2022-09-15 00:00:00 \n', 'OK\n']],
  1663200000.012726],
 [['SE0>csp txrx 4 14 600 0007010100\n',
   ['Answer: 0000\n',
    'Bytes: 2\n',
    'Flags: 0x00 (frag=0, HMAC=0, XTEA=0, RDP=0, CRC32=0)\n',
    'OK\n']],
  1663200000.012726],
 [['SE0>script delay 1000\n', ['OK\n']], 1663200001.012726],
 [['SE0>csp txrx 4 14 600 000A010100\n',
   ['Answer: